In [8]:
import pandas as pd
import matplotlib as plt

df_backstreetboys = pd.read_csv('Dados\Backstreet_Boys_Lyrics_score.csv') 
df_backstreetboys = df_backstreetboys.drop(columns=["Unnamed: 0", "Unnamed: 0.1"]) # Remove colunas desnecessárias

df_arcticmonkeys = pd.read_csv('Dados\lyrics.csv')